Init workspace


In [1]:
!source .venv/bin/activate

In [ ]:
%pip install -qr requirements.txt clearml  # install

import torch
import utils

display = utils.notebook_init()  # checks

Clone Dataset


In [ ]:
!clearml-data get \
--id 63c8e9373cd44d278ebfe693a575387f \
--copy ../datasets/yolo \
--overwrite

Train


In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/test \
--name train_bs32_ep75 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 75 \
--img 736 \
--cache

In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/Evolution \
--name train_bs32_ep30 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 30 \
--img 736 \
--cache \
--evolve \
--resume

Export


In [ ]:
!python export.py \
--data yolo.yaml \
--weights YOLOv5-Pill-Counter/test/train_100/weights/best.pt \
--include tfjs \
--imgsz 736

In [ ]:
# YOLOv5 TensorFlow.js export
import re
import subprocess
from pathlib import Path

weights_file = Path(
    "/home/czack913/Code/yolov5/YOLOv5-Pill-Counter/test/train_50/weights/best.pt"
)

f = str(weights_file).replace(".pt", "_web_model")  # js dir
f_pb = weights_file.with_suffix(".pb")  # *.pb path
f_json = f"{f}/model.json"  # *.json path
args = [
    "tensorflowjs_converter",
    "--input_format=tf_frozen_model",
    "--output_format=tfjs_graph_model",
    "--weight_shard_size_bytes=60000000",
    "--output_node_names=Identity,Identity_1,Identity_2,Identity_3",
    str(f_pb),
    f,
]
subprocess.run([arg for arg in args if arg], check=True)

json = Path(f_json).read_text()
with open(f_json, "w") as j:  # sort JSON Identity_* in ascending order
    subst = re.sub(
        r'{"outputs": {"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}}}',
        r'{"outputs": {"Identity": {"name": "Identity"}, '
        r'"Identity_1": {"name": "Identity_1"}, '
        r'"Identity_2": {"name": "Identity_2"}, '
        r'"Identity_3": {"name": "Identity_3"}}}',
        json,
    )
    j.write(subst)

Validate


In [ ]:
!python val.py \
--weights YOLOv5-Pill-Counter/test/train_100/weights/best.pt \
--data 640x640.yaml \
--img 736

Detect


In [ ]:
!python detect.py \
--weights /home/czack913/Code/yolov5/YOLOv5-Pill-Counter/test/train_100/weights/best.pt \
--source /home/czack913/Code/datasets/resized/padded/images/test/ \
--img 736

Pad

In [3]:
from PIL import Image
import os

IN_DIR = "../datasets/resized/original"
OUT_DIR = "../datasets/resized/padded/images"

os.makedirs(IN_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

TARGET_SIZE = (736, 736)

counter = 0
max_counter = 10

for filename in os.listdir(IN_DIR):
    if filename.lower().endswith((".jpg", ".jpeg")) and counter < max_counter:
        counter += 1

        input_path = os.path.join(IN_DIR, filename)
        output_path = os.path.join(OUT_DIR, filename)

        image = Image.open(input_path)
        original_size = image.size
        # padding_width = max(0, TARGET_SIZE[0] - original_size[0])
        # padding_height = max(0, TARGET_SIZE[1] - original_size[1])
        # anchor_position = (padding_width // 2, padding_height // 2)
        padded_image = Image.new("RGB", TARGET_SIZE, color="white")
        padded_image.paste(image, (0, 0))
        padded_image.save(output_path)

Reformat Annotations

In [ ]:
import os

ORIGINAL_SIZE = (736, 736)
TARGET_SIZE = (640, 640)

FACTOR0 = TARGET_SIZE[0] / ORIGINAL_SIZE[0]
FACTOR1 = TARGET_SIZE[1] / ORIGINAL_SIZE[1]

INPUT_DIR = "/home/czack913/Code/datasets/resized/temp/in"
OUTPUT_DIR = "/home/czack913/Code/datasets/resized/temp/out"

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

counter = 0
MAX_COUNTER = 15

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith(".txt") and counter < MAX_COUNTER:
        counter += 1

        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)

        with open(input_path, "r") as input_file:
            lines = input_file.readlines()

        with open(output_path, "w") as output_file:
            for line in lines:
                if not line.strip():
                    continue

                values = line.split()

                # Check if the line has the expected number of values (5)
                if len(values) != 5:
                    print(f"Skipping line: {line}")
                    continue

                # Extract values
                class_label, x_center, y_center, width, height = map(float, values)

                # Scale bounding box coordinates
                x_center *= FACTOR0
                y_center *= FACTOR1
                width *= FACTOR0
                height *= FACTOR1

                # Write the scaled annotation to the output file
                output_file.write(
                    f"{int(class_label)} {x_center} {y_center} {width} {height}\n"
                )

Reformat File Names

In [9]:
import os
import shutil

INPUT_DIR = "/home/czack913/Code/datasets/resized/temp/in"
OUTPUT_DIR = "/home/czack913/Code/datasets/resized/temp/out"

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

counter = 0
MAX_COUNTER = 15

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".jpg", ".jpeg")) and counter < MAX_COUNTER:
        counter += 1

        input_path = os.path.join(INPUT_DIR, filename)

        ts_part = filename.split("_")[1]
        part = ts_part[:14]

        output_path = os.path.join(OUTPUT_DIR, f"images_{part}.jpeg")

        shutil.copy(input_path, output_path)

In [11]:
from PIL import Image
import os

INPUT_DIR = "/home/czack913/Code/datasets/resized/original"

os.makedirs(INPUT_DIR, exist_ok=True)

unique_sizes = {}

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".jpg", ".jpeg")):
        file_path = os.path.join(INPUT_DIR, filename)

        try:
            # Open the image and get its size
            with Image.open(file_path) as img:
                width, height = img.size
                size_key = f"{width}x{height}"

                # Print size if it's unique
                if size_key not in unique_sizes:
                    print(f"{filename}: {size_key}")
                    unique_sizes[size_key] = True
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

print(unique_sizes)

images_20230429162856.jpeg: 720x720
{'720x720': True}
